# UNData API Exercise #

### Cavin Jacobson ###

**In this exercise, you'll redo the data gathering phase of the UNData Exploration project by using APIs instead of downloading csv files.**

**You'll make use of the World Bank Indicators API. Note that this API does not require an API key. Before attempting the exercise, it would be a good idea to skim through the Documentation page and to check out the Basic Call Structure article.**

In [4]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import json

1. Use the API to get all available data for the GDP per capita, PPP (constant 2017 international $) indicator. Hint: this indicator has code "NY.GDP.PCAP.PP.KD". Adjust the query parameters so that you can retrieve all available rows. Convert the results to a DataFrame.

In [66]:
params = {
    'format': 'json',
    'per_page': 20000
}

In [7]:
endpointgdp = 'https://api.worldbank.org/v2/country/all/indicator/NY.GDP.PCAP.PP.KD'

response = requests.get(endpointgdp, params = params)

In [111]:
gdp_norm = (
    pd.json_normalize(response.json()[1])
    .drop(
        columns = {'unit', 'obs_status', 'country.id', 'indicator.id', 'indicator.value', 'countryiso3code', 'decimal'})
    .rename
        (columns = {'value': 'GDP Per Capita', 'date': 'Year', 'country.value': 'Country'})
)

In [115]:
gdp_norm.sample(3)

,Year,GDP Per Capita,Country
12880,2007,3367.548977,Papua New Guinea
82,2005,3824.095862,Africa Western and Central
10106,1965,NaN,Lao PDR


2. Now, use the API to get all available data for Life expectancy at birth, total (years). This indicator has code "SP.DYN.LE00.IN". Again, convert the results to a DataFrame.

In [83]:
endpointle = 'https://api.worldbank.org/v2/country/all/indicator/SP.DYN.LE00.IN'

response_le = requests.get(endpointle, params = params)

In [85]:
le_res= response_le.json()

In [77]:
le_norm = (
    pd.json_normalize(response_le.json()[1])
    .drop(
        columns = {'unit', 'obs_status', 'country.id', 'indicator.id', 'indicator.value', 'countryiso3code', 'decimal'})
    .rename(
        columns = {'value': 'Life Expectancy', 'date': 'Year', 'country.value': 'Country'})
)

In [15]:
le_norm.head(3)

,Year,Life Expectancy,Country
0,2023,NaN,Africa Eastern and Southern
1,2022,62.899031,Africa Eastern and Southern
2,2021,62.454590,Africa Eastern and Southern


3. Merge the two results DataFrames together. You may want to rename or drop columns prior to merging.

In [17]:
gdp_le_df = pd.merge(gdp_norm[['Year', 'Country', 'GDP Per Capita']], le_norm)

In [109]:
gdp_le_df.sample(5)

,Year,Country,GDP Per Capita,Life Expectancy
3115,1980,World,NaN,62.233617
10780,1995,Malawi,1186.693884,43.641000
8568,1967,Haiti,NaN,46.125000
5602,1989,Chad,NaN,45.875000
5212,1995,Cabo Verde,2808.551368,64.093000


4. You can also get more information about the available countries (region, capital city, income level classification, etc.) by using the Country API. Use this API to pull in all available data. Merge this with your other datasets. Use this to now remove the rows that correspond to regions and not countries.

In [87]:
endpoint_country = 'https://api.worldbank.org/v2/country/all'

response_country = requests.get(endpoint_country, params = params)

In [89]:
country_res = response_country.json()

In [91]:
country_norm =  pd.json_normalize(country_res[1])

In [93]:
country_norm = country_norm.rename(columns = {'name': 'Country'})

In [95]:
gdp_le_df_country = pd.merge(gdp_le_df, country_norm)

In [97]:
gdp_le_df_country = gdp_le_df_country.drop(columns = {'region.value', 'region.id', 'adminregion.value', 'region.iso2code})

In [117]:
gdp_le_df_country.sample(3)

,Year,Country,GDP Per Capita,Life Expectancy,id,iso2Code,capitalCity,longitude,latitude,region.iso2code,adminregion.id,adminregion.iso2code,incomeLevel.id,incomeLevel.iso2code,incomeLevel.value,lendingType.id,lendingType.iso2code,lendingType.value
14272,2023,Somalia,1452.357532,NaN,SOM,SO,Mogadishu,45.3254,2.07515,ZG,SSA,ZF,LIC,XM,Low income,IDX,XI,IDA
12850,1973,Paraguay,NaN,63.066,PRY,PY,Asuncion,-57.6362,-25.3005,ZJ,LAC,XJ,UMC,XT,Upper middle income,IBD,XF,IBRD
6292,2003,Cyprus,39337.535156,77.788,CYP,CY,Nicosia,33.3736,35.1676,Z7,,,HIC,XD,High income,LNX,XX,Not classified


## Bonus Questions ##

1. Adjust your request so that it returns data just for the United States 

In [29]:
endpoint_us = 'https://api.worldbank.org/v2/country/us?format=json'

2. Adjust your request so that it returns data just for the United States for the year 2021.

In [31]:
endpoint_us_2021 = 'https://api.worldbank.org/v2/country/us?date=2021/format=json'

In [32]:
endpoint_us_2021_res = requests.get(endpoint_us_2021)

3. Adjust your request so that it returns data just for the United States for the years 2000 through 2021.

In [55]:
endpoint_us_2000_2021 = 'https://api.worldbank.org/v2/country/us?date=2000:2021/format=json'

4. Adjust your request so that it returns data for the United States and Canada for the years 2000 through 2021.

In [59]:
endpoint_us_ca = 'https://api.worldbank.org/v2/country/us;ca?date=2000:2021/format=json'

5. If you haven't already done so and you would like to get some additional practice using loops, use the page parameter in order to pull all records. Do not change the value of the per_page parameter. You will likely need to utilize a loop of some kind in order to pull all records.

6. The endpoint "http://api.worldbank.org/v2/indicator" will return information about all available indicators. See Indicator API Queries. Use this endpoint to find the code for the "Public Expenditure on Education (% GDP)" indicator. 